In [ ]:
# A_AugmentationScaleAndUpdateAnnotationsANdPreview
# Scale images and update annotations 
# Scale the entire image and overlay it onto the original image size. 
#  Update the annotations based on the scaled coordinates of the bounding boxes. 
# Scaling the Image:
# Scaled image by a specified factor using cv2.resize.
# Overlay the scaled image onto the original-sized canvas.
# Adjust and scale annotations using the same factor.
# Clip Bounding box dimensions to the original image dimensions????

# Shifted the overlaid image upwards (offset_y) so it lowerpart fits within the original dimensions.
# Adjust bounding box cy values by subtracting the vertical offset.
# Bounding Box Format Handling:
# YOLO annotations define bounding boxes relative to the image dimensions. 
# When scaling, boxes remain inside the image, with adjusted dimensions, and width and height are scaled consistently.
# Offset Calculation:
# Offsets are calculated correctly if the scaled image is smaller than the original. 
# The offset_x and offset_y values should consider both positive and negative adjustments.

# Handling of Bounding Boxes:
# Calculated absolute bounding box coordinates (x_min, y_min, x_max, y_max) to ensure valid and clipped bounding boxes.
# Check invalid boxes and skip them
# Offset logic handles both positive and negative values 
# new_annotations.append skips invalid boxes.
# YOLO annotations are formatted with six decimal places.

# Log a confirmation mesage for each processed image
# Print a mssg if an annotation is missing and skip the image

# Handles a folder of images and corresponding annotations.
# Save scaled images and updated annotation to the output folder.

# Further info: Calcualte horizontal (offset_x) and vertical (offset_y) offsets to center the scaled image within the original dimensions.
# Clip scaled areas beyond the original image dimensions 
# Adjust annotations to reflect the scaled image's new positions and dimensions.

# new_annotations.append(f"{class_id} {cx} {cy} {bw} {bh}\n")
# Update the annotation with six decimal places
# new_annotations.append(f"{class_id:.6f} {cx:.6f} {cy:.6f} {bw:.6f} {bh:.6f}\n")

# output_images_folder = os.path.join(output_folder, "images")
#output_annotations_folder = os.path.join(output_folder, "annotations")
    
#   output_images_folder = output_folder
#   output_annotations_folder = output_folder

import os
import cv2
import matplotlib.pyplot as plt  # For preview
import numpy as np

def load_class_names(obj_names_path):
    """
    Loads class names from the obj.names file.
    Args:
        obj_names_path (str): Path to the obj.names file.
    Returns:
        list: A list of class names.
    """
    if not os.path.exists(obj_names_path):
        raise FileNotFoundError(f"obj.names file not found at {obj_names_path}")
    with open(obj_names_path, "r") as f:
        class_names = f.read().strip().split("\n")
    return class_names


def generate_output_path(base_path, file_name, suffix, extension=None):
        base_name = os.path.splitext(file_name)[0]
        ext = extension or os.path.splitext(file_name)[1]
        return os.path.join(base_path, f"{base_name}_{suffix}{ext}")


def generate_scale_string(scale_factor):
    """
    Converts the scale factor into a string with a pattern 'scale_x_y' 
    where the decimal point is replaced with an underscore.

    Args:
        scale_factor (float): The scale factor.

    Returns:
        str: The scale factor string in the format 'scale_x_y'.
    """
    return f"scale_{str(scale_factor).replace('.', '_')}"


def center_image_and_update_annotations(image_path, annotation_path, output_image_path, output_annotation_path):
    """
    Centers the image based on bounding boxes of classes 0, 1, 2, and 3,
    and updates all annotations with the computed shift.
    
    Args:
        image_path (str): Path to the input image.
        annotation_path (str): Path to the corresponding annotation file.
        output_image_path (str): Path to save the centered image.
        output_annotation_path (str): Path to save the updated annotations.
    """
    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"Image not found: {image_path}")

    original_h, original_w = image.shape[:2]

    # Initialize variables for bounding box calculation
    x_min, y_min, x_max, y_max = 1.0, 1.0, 0.0, 0.0  # Normalized bounds
    valid_bounding_boxes = False

    # Read annotations and calculate center of relevant bounding boxes (classes 0, 1, 2, 3)
    with open(annotation_path, "r") as annotation_file:
        for annotation in annotation_file:
            class_id, cx, cy, bw, bh = map(float, annotation.split())
            if int(class_id) > 3:  # Only consider flag classes 0, 1, 2, 3 for centering
                continue

            valid_bounding_boxes = True
            x_min = min(x_min, cx - bw / 2)
            y_min = min(y_min, cy - bh / 2)
            x_max = max(x_max, cx + bw / 2)
            y_max = max(y_max, cy + bh / 2)

    # Skip centering if no relevant bounding boxes are found
    if not valid_bounding_boxes:
        print(f"No valid bounding boxes (classes 0-3) in {annotation_path}. Skipping centering.")
        cv2.imwrite(output_image_path, image)
        shutil.copy(annotation_path, output_annotation_path)
        return

    # Convert normalized coordinates to pixel values
    box_center_x = int(((x_min + x_max) / 2) * original_w)
    box_center_y = int(((y_min + y_max) / 2) * original_h)

    # Calculate shift to center the bounding box center in the image
    shift_x = (original_w // 2) - box_center_x
    shift_y = (original_h // 2) - box_center_y

    # Apply the shift to the image
    transformation_matrix = np.float32([[1, 0, shift_x], [0, 1, shift_y]])
    shifted_image = cv2.warpAffine(image, transformation_matrix, (original_w, original_h), borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0))

    # Save the shifted image
    output_directory = os.path.dirname(output_image_path)
    os.makedirs(output_directory, exist_ok=True)
    cv2.imwrite(output_image_path, shifted_image)

    # Update annotations
    new_annotations = []
    with open(annotation_path, "r") as annotation_file:
        for annotation in annotation_file:
            class_id, cx, cy, bw, bh = map(float, annotation.split())

            # Adjust bounding box centers
            cx = (cx * original_w + shift_x) / original_w
            cy = (cy * original_h + shift_y) / original_h

            # Clamp the bounding box coordinates to ensure they stay within bounds
            x_min = max(0, cx - bw / 2)
            y_min = max(0, cy - bh / 2)
            x_max = min(1, cx + bw / 2)
            y_max = min(1, cy + bh / 2)

            # Recalculate bounding box center, width, and height
            cx = (x_min + x_max) / 2
            cy = (y_min + y_max) / 2
            bw = x_max - x_min
            bh = y_max - y_min

            # Only append the bounding box if it still has valid dimensions
            if bw > 0 and bh > 0:
                new_annotations.append(f"{int(class_id)} {cx:.6f} {cy:.6f} {bw:.6f} {bh:.6f}\n")
            else:
                print(f"Clamped box for class {class_id} has zero area and will not be included.")

    # Save updated annotations
    with open(output_annotation_path, "w") as annotation_file:
        annotation_file.writelines(new_annotations)

    #print(f"Centered image and annotations saved to: {output_image_path}, {output_annotation_path}")


def scale_image_and_update_annotations(src, ann_path, output_image_path, output_ann_path, scale_factor):
    """
    Scales the image, centers the overlay within the original dimensions, and updates annotations.
    """
    # Load the image
    image = cv2.imread(src)
    if image is None:
        raise FileNotFoundError(f"Image not found: {src}")

    original_h, original_w = image.shape[:2]

    # Scale the image
    scaled_h, scaled_w = int(original_h * scale_factor), int(original_w * scale_factor)
    scaled_image = cv2.resize(image, (scaled_w, scaled_h))

    # Calculate offsets to center the scaled image
    offset_y = max(0, (scaled_h - original_h) // 2)
    offset_x = max(0, (scaled_w - original_w) // 2)

    # Create a blank canvas for the original image size
    new_image = np.zeros_like(image)  # Blank canvas for the scaled crop

    # Extract the region from the scaled image that fits the original size
    scaled_crop = scaled_image[
        offset_y:offset_y + original_h,
        offset_x:offset_x + original_w
    ]
    new_image[:scaled_crop.shape[0], :scaled_crop.shape[1]] = scaled_crop

    # Ensure the output directory exists
    output_directory = os.path.dirname(output_image_path)
    os.makedirs(output_directory, exist_ok=True)

    # Save the scaled image
    cv2.imwrite(output_image_path, new_image)

    # Update annotations
    new_annotations = []
    with open(ann_path, "r") as ann_file:
        for annotation in ann_file:
            class_id, cx, cy, bw, bh = map(float, annotation.split())

            # Convert annotation to absolute values
            cx_abs = cx * original_w
            cy_abs = cy * original_h
            bw_abs = bw * original_w
            bh_abs = bh * original_h

            # Scale and adjust the coordinates
            cx_abs = (cx_abs * scale_factor) - offset_x
            cy_abs = (cy_abs * scale_factor) - offset_y
            bw_abs *= scale_factor
            bh_abs *= scale_factor

            # Calculate the bounding box coordinates
            x_min = max(0, cx_abs - bw_abs / 2)
            y_min = max(0, cy_abs - bh_abs / 2)
            x_max = min(original_w, cx_abs + bw_abs / 2)
            y_max = min(original_h, cy_abs + bh_abs / 2)

            # Clamp bounding box dimensions to image boundaries
            x_min = max(0, x_min)
            y_min = max(0, y_min)
            x_max = min(original_w, x_max)
            y_max = min(original_h, y_max)

            # Recalculate bounding box center, width, and height
            cx_abs = (x_min + x_max) / 2
            cy_abs = (y_min + y_max) / 2
            bw_abs = x_max - x_min
            bh_abs = y_max - y_min

            # Skip bounding boxes with zero or negative dimensions
            if bw_abs <= 0 or bh_abs <= 0:
                print(f"Clamped box for class {class_id} has zero area and will not be included.")
                continue

            # Convert back to relative values
            cx = cx_abs / original_w
            cy = cy_abs / original_h
            bw = bw_abs / original_w
            bh = bh_abs / original_h

            # Append updated annotation
            new_annotations.append(f"{int(class_id)} {cx:.6f} {cy:.6f} {bw:.6f} {bh:.6f}\n")

    # Save updated annotations
    with open(output_ann_path, "w") as annotation_file:
        annotation_file.writelines(new_annotations)

    #print(f"Scaled image saved to: {output_image_path}")
    #print(f"Updated annotations saved to: {output_ann_path}")

def shift_image_and_update_annotations(
        image_path, 
        annotation_path, 
        output_image_path, 
        output_annotation_path, 
        shift_x_pc=0.1, 
        shift_y_pc=0.1):
    """
    Shifts the image horizontally/vertically and updates annotations accordingly.
    
    Args:
        image_path (str): Path to the input image.
        annotation_path (str): Path to the corresponding annotation file.
        output_image_path (str): Path to save the shifted image.
        output_annotation_path (str): Path to save the updated annotations.
        shift_x_pc (float): Fraction of the image width to shift (-1 to 1).
        shift_y_pc (float): Fraction of the image height to shift (-1 to 1).
    """
    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"Image not found: {image_path}")
    
    original_h, original_w = image.shape[:2]

    # Convert percentage shifts to pixel shifts
    shift_x = int(original_w * shift_x_pc)
    shift_y = int(original_h * shift_y_pc)

    # Create a blank canvas for the shifted image
    new_image = np.zeros_like(image)

    # Handle vertical shift
    if shift_y > 0:  # Shift down
        new_image[shift_y:, :] = image[:-shift_y, :]
    elif shift_y < 0:  # Shift up
        new_image[:shift_y, :] = image[-shift_y:, :]
    else:
        new_image[:, :] = image[:, :]  # No vertical shift

    # Handle horizontal shift
    if shift_x > 0:  # Shift right
        new_image[:, shift_x:] = new_image[:, :-shift_x]
    elif shift_x < 0:  # Shift left
        new_image[:, :shift_x] = new_image[:, -shift_x:]
    else:
        new_image[:, :] = new_image[:, :]  # No horizontal shift

    # Save the shifted image
    cv2.imwrite(output_image_path, new_image)

    # Update annotations
    new_annotations = []
    with open(annotation_path, "r") as annotation_file:
        for annotation in annotation_file:
            class_id, cx, cy, bw, bh = map(float, annotation.split())

            # Adjust bounding box centers
            cx += shift_x_pc
            cy += shift_y_pc

            # Validate if the box stays in the valid range
            if (cx - bw / 2 < 0 or cx + bw / 2 > 1 or
                cy - bh / 2 < 0 or cy + bh / 2 > 1):
                continue  # Skip bounding boxes that go out of bounds

            # Append updated annotation
            new_annotations.append(f"{int(class_id)} {cx:.6f} {cy:.6f} {bw:.6f} {bh:.6f}\n")

    # Save updated annotations
    with open(output_annotation_path, "w") as annotation_file:
        annotation_file.writelines(new_annotations)

    #print(f"Shifted image saved to: {output_image_path}")
    #print(f"Updated annotations saved to: {output_annotation_path}")


def process_folderOLD(src_fld, tgt_fld, job, scale_factor, shift_x_pc=0, shift_y_pc=0, scaling=False, shifting=False, center=False):
    """
    Processes all images and annotations in a folder.
    """
    print(f"Beginning to process {job}...scale factor is: {scale_factor}")
    
    #print(f"tgt: {tgt_fld}, job: {job}")
    #print(f"src: {src_fld}, job: {job}")

    src = os.path.join(src_fld, job)
    tgt = os.path.join(tgt_fld, job)

    #print(f"tgt: {tgt}, job: {job}")
    #print(f"src: {src}, job: {job}")

    # output_images_folder = tgt #os.path.join(output_folder, images_sub_folder)
    #output_ann_folder = tgt #os.path.join(output_folder, annotations_sub_folder)
    #os.makedirs(output_images_folder, exist_ok=True)
    #os.makedirs(output_annotations_folder, exist_ok=True)
    #output_images_folder = os.path.join(tgt, job)

    #os.makedirs(output_folder, exist_ok=True)
    image_files = [f for f in os.listdir(src_fld) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    os.makedirs(tgt, exist_ok=True)

    for image_file in image_files:
        #image_path = os.path.join(images_folder, image_file)
        src_image_path = os.path.join(src, image_file)
        #print(f"src imgpath {src_image_path}")
        src_ann_path = os.path.join(src, os.path.splitext(image_file)[0] + ".txt")
        #annotation_path = os.path.join(annotations_folder, os.path.splitext(image_file)[0] + ".txt")
        if not os.path.exists(src_ann_path):
            print(f"Annotation missing for image: {image_file}, skipping.")
            continue
        if center:
            #output_image_path_center = os.path.join(tgt, f"{image_file}_centered")
            # Append "_centered" to the image file name and retain the extension
            output_image_path_center = os.path.join(tgt, f"{os.path.splitext(image_file)[0]}_centered{os.path.splitext(image_file)[1]}")
             # Append "_centered" for the annotation file as well
            output_ann_path_center = os.path.join(tgt, f"{os.path.splitext(image_file)[0]}_centered.txt")
            # output_ann_path_center = os.path.join(tgt, f"{os.path.splitext(image_file)[0]}_centered.txt")
            center_image_and_update_annotations(src_image_path, src_ann_path, output_image_path_center, output_ann_path_center)
            #Pass items from this into the next operation  
            src_image_path = output_image_path_center
            src_ann_path = output_ann_path_center
        
        if shifting:
            output_image_path_shift = os.path.join(tgt, f"{image_file}_shifted")
            output_ann_path_shift = os.path.join(tgt, f"{os.path.splitext(image_file)[0]}_shifted.txt")
            shift_image_and_update_annotations(src_image_path, src_ann_path, output_image_path_shift, output_ann_path_shift, shift_x_pc, shift_y_pc)
            # make these the new input paths for further work 
            #src_image_path = output_image_path_shift
            #src_ann_path = output_ann_path_shift
            # Pass items from this into the scaling function 
            src_image_path = output_image_path_shift
            src_ann_path = output_ann_path_shift
        
        if scaling:
            scale_str = str(scale_factor).replace('.', '_')
            #output_images_folder = f("{output_images_folder}_{scale_str}")
            output_image_path_scale = os.path.join(tgt, f"{os.path.splitext(image_file)[0]}_scaled_{scale_str}")
            output_ann_path_scale =   os.path.join(tgt, f"{os.path.splitext(image_file)[0]}_scaled_{scale_str}.txt")
            #output_image_path_scale = os.path.join(tgt, "{image_file}_scaled_{scale_str}")
            #output_ann_path_scale = os.path.join(tgt, "{os.path.splitext(image_file)[0]}_scaled_{scale_str}.txt")
            scale_image_and_update_annotations(src_image_path, src_ann_path, output_image_path_scale,  output_ann_path_scale, scale_factor)
            #print(f" source image path:  {src_image_path}")
            #print(f" source annotation path: {src_ann_path}")
            #print(output_image_path_scale)
            #print(output_ann_path_scale)
            #print(1.5)
    
    print(f"Augmentation cycle completed for {job}...")
    if shifting:
        print(f" - Vertical/horizontal shift applied (x: {shift_x_pc}, y: {shift_y_pc})") #f\n
    if scaling:
        print(f" - Images scaled - scale factor: {scale_factor}")
    if center:
        print("  - Images centered based on bounding boxes.")


def cleanup_centered_files(base_tgt):
    """
    Deletes all files containing '_centered_' in their filenames
    within the subfolders of base_tgt.

    Args:
        base_tgt (str): Path to the base directory.
    """
    for root, _, files in os.walk(base_tgt):
        for file in files:
            if '_centered_' in file:
                file_path = os.path.join(root, file)
                try:
                    os.remove(file_path)
                    print(f"Deleted: {file_path}")
                except Exception as e:
                    print(f"Failed to delete {file_path}: {e}")



def process_folder(src_fld, tgt_fld, job, scale_factor, shift_x_pc=0, shift_y_pc=0, scaling=False, shifting=False, center=False):
    """
    Processes all images and annotations in a folder.
    """
    print(f"Beginning to process {job}...scale factor is: {scale_factor}")
    
    src = os.path.normpath(os.path.join(src_fld, job))
    tgt = os.path.normpath(os.path.join(tgt_fld, job))
    os.makedirs(tgt, exist_ok=True)

    image_files = [
        f for f in os.listdir(src)
        if f.lower().endswith(('.png', '.jpg', '.jpeg')) and 
        os.path.exists(os.path.join(src, os.path.splitext(f)[0] + ".txt"))
    ]

    for image_file in image_files:
        src_image_path = os.path.join(src, image_file)
        src_ann_path = os.path.join(src, os.path.splitext(image_file)[0] + ".txt")

        current_image_path = src_image_path
        current_ann_path = src_ann_path

        if center:
            output_image_path_center = generate_output_path(tgt, image_file, "centered")
            output_ann_path_center = generate_output_path(tgt, image_file, "centered", ".txt")
            center_image_and_update_annotations(current_image_path, current_ann_path, output_image_path_center, output_ann_path_center)
            current_image_path, current_ann_path = output_image_path_center, output_ann_path_center

        #if shifting:
        #    output_image_path_shift = generate_output_path(tgt, image_file, "shifted")
        #    output_ann_path_shift = generate_output_path(tgt, image_file, "shifted", ".txt")
        #    shift_image_and_update_annotations(current_image_path, current_ann_path, output_image_path_shift, output_ann_path_shift, shift_x_pc, shift_y_pc)
        #    current_image_path, current_ann_path = output_image_path_shift, output_ann_path_shift

        if scaling:
            output_image_path_scale = generate_output_path(tgt, image_file, f"scaled_{str(scale_factor).replace('.', '_')}")
            output_ann_path_scale = generate_output_path(tgt, image_file, f"scaled_{str(scale_factor).replace('.', '_')}", ".txt")
            scale_image_and_update_annotations(current_image_path, current_ann_path, output_image_path_scale, output_ann_path_scale, scale_factor)

    # clean up the intermediate atage files with '_centered_' in the filename
    cleanup_centered_files(tgt)

    print(f"Processing completed for job: {job}")
 
def preview_images(image_folder, annotation_folder, obj_names_path, preview_count=3):
    """
    Displays images with bounding boxes and class labels.
    Args:
        image_folder (str): Folder containing images.
        annotation_folder (str): Folder containing annotations.
        obj_names_path (str): Path to the obj.names file.
        preview_count (int): Number of images to preview.
    """
    class_names = load_class_names(obj_names_path)
    image_files = sorted([f for f in os.listdir(image_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])

    for image_file in image_files[:preview_count]:
        image_path = os.path.join(image_folder, image_file)
        annotation_path = os.path.join(annotation_folder, os.path.splitext(image_file)[0] + ".txt")

        if not os.path.exists(annotation_path):
            continue

        image = cv2.imread(image_path)
        preview = image.copy()

        with open(annotation_path, "r") as f:
            annotations = f.readlines()

        for ann in annotations:
            class_id, cx, cy, bw, bh = map(float, ann.split())
            x_center = int(cx * image.shape[1])
            y_center = int(cy * image.shape[0])
            box_width = int(bw * image.shape[1])
            box_height = int(bh * image.shape[0])
            x1 = int(x_center - box_width / 2)
            y1 = int(y_center - box_height / 2)
            x2 = int(x_center + box_width / 2)
            y2 = int(y_center + box_height / 2)
            cv2.rectangle(preview, (x1, y1), (x2, y2), (0, 255, 0), 2)
            label = class_names[int(class_id)]
            cv2.putText(preview, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 0, 0), 1)

        plt.figure(figsize=(10, 10))
        plt.imshow(cv2.cvtColor(preview, cv2.COLOR_BGR2RGB))
        plt.title(f"Preview of {image_file}")
        plt.axis("off")
        plt.show()


# Usage Section
if __name__ == "__main__":
    #base_folder = r'D:\FlagDetectionDatasets\ExportedDatasetsReduced\AugTest'
    #base_folder = r'D:\FlagDetectionDatasets\Augmentation

    #output_folder = r'D:/FlagDetectionDatasets/ExportedDatasetsReduced/Augmentation/Job_30_scaled'
    #annotations_sub_folder = "obj_train_data" 
    #images_sub_folder = "obj_train_data"  
    #obj_names_path = os.path.join(base_folder, "obj.names")  # Path to obj.names

    #base_folder = f'D:\FlagDetectionDatasets\Augmentation\scaled\source\Job_88'

    #base_folder = r'D:\FlagDetectionDatasets\Augmentation\scaled\source\Job_88'
    
    #base_folder = r'D:\FlagDetectionDatasets\Augmentation\scaled\source\test'
    #base_folder = r'D:\FlagDetectionDatasets\Augmentation\scaled\source\test\1_shifted'
    #output_folder = r'D:\FlagDetectionDatasets\Augmentation\scaled\augmented\Job_88'
    
    # Parameters
    #shift_percent = 0 #-.25  # Positive for right shift, negative for left shift

    # Construct paths
    #source_images_folder = os.path.join(base_folder, images_sub_folder)
    #source_annotations_folder = os.path.join(base_folder, annotations_sub_folder)
    #source_images_folder = base_folder
    #source_annotations_folder = base_folder # annotations_sub_folder

    # Parametersnorm
    scale_factor = 1.7 #2.3 # 88 / job_36 / job_126 1.5 # 1.2
    shift_x_pc = 0 #-.3  #20%
    shift_y_pc = 0 #-.04 #0 #-.2 #.09 #.06 # minus pushes img up 
    preview_count = 3

    # nEXT RUN 
    #process_folder(base_src,base_tgt,job='Job_7',scale_factor=1.8,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    # 23
    # Job 22: scalefactor 1.9, shift_y_pc : .06    scaling = True,   shifting = True 
    
    #base_src = r'D:/FlagDetectionDatasets/ExportedDatasetsReduced'

    #base_tgt = r'D:/FlagDetectionDatasets/Augmentation/scaled/augmented'

    #base_src = os.path.normpath('D:/FlagDetectionDatasets/ExportedDatasetsExtracted')
    base_src = os.path.normpath('D:/FlagDetectionDatasets/ExportedDatasetsReduced')
    base_tgt = os.path.normpath('D:/FlagDetectionDatasets/Augmentation/scaled/augmented')
  
    #process_folder(base_src,base_tgt,job='Job_88',scale_factor=2.2,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    # REDO process_folder(base_src,base_tgt,job='Job_23',scale_factor=1.8,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    #process_folder(base_src,base_tgt,job='Job_7',scale_factor=2.4,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    #process_folder(base_src,base_tgt,job='Job_11',scale_factor=2.0,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    # ry flag at home 
    #process_folder(base_src,base_tgt,job='Job_12',scale_factor=2.0,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    #process_folder(base_src,base_tgt,job='Job_17',scale_factor=1.4,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    #process_folder(base_src,base_tgt,job='Job_18',scale_factor=1.4,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)

    # red flag home 
    #process_folder(base_src,base_tgt,job='Job_13',scale_factor=1.4,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    #process_folder(base_src,base_tgt,job='Job_14',scale_factor=1.4,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    #process_folder(base_src,base_tgt,job='Job_15',scale_factor=1.4,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    #process_folder(base_src,base_tgt,job='Job_16',scale_factor=1.2,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)

    # Ex quality blue + red 
    #process_folder(base_src,base_tgt,job='Job_21',scale_factor=1.5,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    #process_folder(base_src,base_tgt,job='Job_22',scale_factor=1.5,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    
    # REDO process_folder(base_src,base_tgt,job='Job_23',scale_factor=2.2,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    
    #process_folder(base_src,base_tgt,job='Job_27',scale_factor=1.4,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    #process_folder(base_src,base_tgt,job='Job_28',scale_factor=1.3,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    #process_folder(base_src,base_tgt,job='Job_29',scale_factor=1.4,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)

    #process_folder(base_src,base_tgt,job='Job_30',scale_factor=1.5,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    #process_folder(base_src,base_tgt,job='Job_31',scale_factor=1.4,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    #process_folder(base_src,base_tgt,job='Job_32',scale_factor=1.5,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    #process_folder(base_src,base_tgt,job='Job_36',scale_factor=1.6,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    #process_folder(base_src,base_tgt,job='Job_37',scale_factor=1.7,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    #process_folder(base_src,base_tgt,job='Job_41',scale_factor=1.5,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    
    process_folder(base_src,base_tgt,job='Job_42',scale_factor=1.4,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    process_folder(base_src,base_tgt,job='Job_48',scale_factor=1.5,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    process_folder(base_src,base_tgt,job='Job_50',scale_factor=1.4,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    
    #process_folder(base_src,base_tgt,job='Job_51',scale_factor=1.8,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    
    #process_folder(base_src,base_tgt,job='Job_54',scale_factor=2.5,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    #process_folder(base_src,base_tgt,job='Job_55',scale_factor=2.5,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    #process_folder(base_src,base_tgt,job='Job_55',scale_factor=3.0,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    # SKIP process_folder(base_src,base_tgt,job='Job_57',scale_factor=3.0,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    #process_folder(base_src,base_tgt,job='Job_59',scale_factor=3.0,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)

    #process_folder(base_src,base_tgt,job='Job_60',scale_factor=3.0,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    #process_folder(base_src,base_tgt,job='Job_61',scale_factor=3.0,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    # sKIP process_folder(base_src,base_tgt,job='Job_65',scale_factor=3.0,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    # SKIP process_folder(base_src,base_tgt,job='Job_66',scale_factor=3.0,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    # skip rocess_folder(base_src,base_tgt,job='Job_67',scale_factor=3.0,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    # skip process_folder(base_src,base_tgt,job='Job_69',scale_factor=3.0,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    #process_folder(base_src,base_tgt,job='Job_70',scale_factor=3.0,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    
    process_folder(base_src,base_tgt,job='Job_72',scale_factor=1.4,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)

    process_folder(base_src,base_tgt,job='Job_76',scale_factor=1.7,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    process_folder(base_src,base_tgt,job='Job_87',scale_factor=3.0,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    #process_folder(base_src,base_tgt,job='Job_88',scale_factor=2.2,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    #process_folder(base_src,base_tgt,job='Job_89',scale_factor=2.2,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    # kip process_folder(base_src,base_tgt,job='Job_95',scale_factor=2.2,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    # SKIP process_folder(base_src,base_tgt,job='Job_96',scale_factor=2.2,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    # skip process_folder(base_src,base_tgt,job='Job_98',scale_factor=2.2,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    process_folder(base_src,base_tgt,job='Job_104',scale_factor=3.0,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    process_folder(base_src,base_tgt,job='Job_105',scale_factor=2.6,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    process_folder(base_src,base_tgt,job='Job_106',scale_factor=2.7,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    # SKIP process_folder(base_src,base_tgt,job='Job_108',scale_factor=2.2,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    # process_folder(base_src,base_tgt,job='Job_109',scale_factor=2.2,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    process_folder(base_src,base_tgt,job='Job_114',scale_factor=2.5,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    process_folder(base_src,base_tgt,job='Job_115',scale_factor=2.2,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    process_folder(base_src,base_tgt,job='Job_116',scale_factor=2.2,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)

    process_folder(base_src,base_tgt,job='Job_117',scale_factor=2.0,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    ## REDO ANNOTATIONS MESSED UP 
    # process_folder(base_src,base_tgt,job='Job_118',scale_factor=1.5,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    process_folder(base_src,base_tgt,job='Job_119',scale_factor=1.7,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    # GOOD process_folder(base_src,base_tgt,job='Job_121',scale_factor=3.0,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    #process_folder(base_src,base_tgt,job='Job_123',scale_factor=1.8,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    process_folder(base_src,base_tgt,job='Job_125',scale_factor=1.9,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    
    process_folder(base_src,base_tgt,job='Job_126',scale_factor=1.5,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    
    process_folder(base_src,base_tgt,job='Job_130',scale_factor=2.2,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    process_folder(base_src,base_tgt,job='Job_143',scale_factor=2.2,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    process_folder(base_src,base_tgt,job='Job_145',scale_factor=1.4,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    # sKIP process_folder(base_src,base_tgt,job='Job_146',scale_factor=2.2,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)

    process_folder(base_src,base_tgt,job='Job_24_filter',scale_factor=1.4,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    process_folder(base_src,base_tgt,job='Job_25_filter',scale_factor=1.4,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)
    process_folder(base_src,base_tgt,job='Job_27_filter',scale_factor=1.4,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)

    process_folder(base_src,base_tgt,job='Job_ssim_24',scale_factor=1.4,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)

    process_folder(base_src,base_tgt,job='Job_ssim_25',scale_factor=1.4,shift_x_pc=shift_x_pc,shift_y_pc=shift_y_pc,scaling = True, shifting = False,  center = True)


    previewjob='Job_23'
    #print(base_src)
    preview_folder=  os.path.join(base_src, previewjob)
    #obj_names_path=  os.path.join(base_src, "obj.names")
    #preview_foldertmp = os.path.join(base_tmp, previewjob)
    #preview_images(preview_foldertmp, preview_foldertmp, obj_names_path, preview_count=10)
    #preview_images(preview_folder, preview_folder, obj_names_path, preview_count=10)

    # Process folder with scaling and shifting (enable/disable as needed)
    #print("Doing my thing on...")
  
    # Job 22 > shift 12 y - scale 
    # Preview scaled images
    #print("\nPreviewing augmented images...")
    #preview_images(
    #    source_images_fodef preview_images(image_folder, annotation_folder, obj_names_path, preview_count=3):#output_folder, #source_images_folder, 
    #    source_images_folder, #output_folder, # source_images_folder, 
    #    obj_names_path='D:\FlagDetectionDatasets\Augmentation\scaled\source\obj.names',
    #    preview_count=preview_count
    #)
    
    # Beginning to process D:\FlagDetectionDatasets\Augmentation\scaled\source\Job_11...scale factor is: 1.7


Beginning to process Job_42...scale factor is: 1.4
Processing completed for job: Job_42
Beginning to process Job_48...scale factor is: 1.5
Processing completed for job: Job_48
Beginning to process Job_50...scale factor is: 1.4
Processing completed for job: Job_50
Beginning to process Job_72...scale factor is: 1.4
Processing completed for job: Job_72
Beginning to process Job_76...scale factor is: 1.7
Processing completed for job: Job_76
Beginning to process Job_87...scale factor is: 3.0
Processing completed for job: Job_87
Beginning to process Job_104...scale factor is: 3.0
Processing completed for job: Job_104
Beginning to process Job_105...scale factor is: 2.6
Processing completed for job: Job_105
Beginning to process Job_106...scale factor is: 2.7
Processing completed for job: Job_106
Beginning to process Job_114...scale factor is: 2.5
Processing completed for job: Job_114
Beginning to process Job_115...scale factor is: 2.2
Processing completed for job: Job_115
Beginning to process J